In [24]:
### LOAD PACKAGES ###
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.cluster import AgglomerativeClustering
from scipy.stats import kruskal,chisquare
import altair as alt
import ugtm
import textwrap
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy 
from scipy.cluster.hierarchy import dendrogram, linkage
import os
from tqdm.notebook import trange, tqdm
from tqdm.auto import tqdm
from itertools import chain
%matplotlib inline


In [25]:
import warnings

# 忽略所有 SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [26]:
def get_fields(fields, data, data_field):
    f = data_field[data_field["field.showcase"].isin(fields) & data_field["field.tab"].str.contains("f\\.\\d+\\.0\\.\\d")].copy()
    f["field"] = pd.Categorical(f["field.showcase"], categories=fields, ordered=True)
    f = f.sort_values("field").reset_index().drop("field", axis=1)
    return f

def get_fields_all(fields, data, data_field):
    f = data_field[data_field["field.showcase"].isin(fields)].copy()
    f["field"] = pd.Categorical(f["field.showcase"], categories=fields, ordered=True)
    f = f.sort_values("field").reset_index().drop("field", axis=1)
    return f

def get_data_fields(fields, data, data_field):
    f = get_fields(fields, data, data_field)
    return data[["eid"]+f["col.name"].to_list()].copy()

def get_data_fields_all(fields, data, data_field):
    f = get_fields_all(fields, data, data_field)
    return data[["eid"]+f["col.name"].to_list()].copy()

In [27]:
def map_column_to_meaning(df, column_name, data_path, file):
    # 读取数据文件
    coding1001 = pd.read_csv(f"{data_path}/{file}", sep="\t")
    
    # 将 coding 列转换为字符串类型
    coding1001['coding'] = coding1001['coding'].astype('str')
    
    # 将列重命名为指定的 column_name
    coding1001.rename(columns={"coding": column_name}, inplace=True)
    
    # 将 column_name 列转换为字符串类型
    df[column_name] = df[column_name].astype('str')
    
    # 创建 code 到 meaning 的映射字典
    code_to_meaning = dict(zip(coding1001[column_name], coding1001['meaning']))
    
    # 使用映射字典替换 column_name 列的值，并将其转换为分类类型
    df[column_name] = df[column_name].map(code_to_meaning).astype('category')

In [29]:
from datetime import datetime, timedelta

def datetime_from_dec_year(dec_year):
    start = dec_year
    year = int(start)
    rem = start - year

    base = datetime(year, 1, 1)
    result = base + timedelta(seconds=(base.replace(year=base.year + 1) - base).total_seconds() * rem)
    #result.strftime("%Y-%m-%d")
    return result.date()

def extract_map_self_reported(data, data_field, code_map):
    pbar = tqdm(total=16)
    ### codes
    fields = ["20002"]; pbar.update(1)
    raw = get_data_fields_all(fields, data, data_field); pbar.update(1)
    col = "noncancer_illness_code_selfreported_f20002"; pbar.update(1)
    temp = pd.wide_to_long(raw, stubnames=[col], i="eid", j="instance_index", sep="_", suffix="\w+").reset_index(); pbar.update(1)
    codes = temp.rename(columns={col:"code"})\
        .assign(code=lambda x: x.code.astype(str))\
        .replace("None", np.nan) \
        .replace("nan", np.nan) \
        .dropna(subset=["code"], axis=0)\
        .assign(code=lambda x: x.code.astype(int)) \
        .merge(code_map, how="left",on="code") \
        .sort_values(["eid", "instance_index"]) \
        .reset_index(drop=True); pbar.update(1)
    
    ### dates
    fields = ["20008"]; pbar.update(1)
    raw = get_data_fields_all(fields, data, data_field); pbar.update(1)
    col="interpolated_year_when_noncancer_illness_first_diagnosed_f20008"; pbar.update(1)
    temp = pd.wide_to_long(raw, stubnames=[col], i="eid", j="instance_index", sep="_", suffix="\w+").reset_index(); pbar.update(1)
    dates = temp.rename(columns={col:"date"})\
        .dropna(subset=["date"], axis=0)\
        .sort_values(["eid", "instance_index"]) \
        .reset_index(drop=True); pbar.update(1)

    dates = dates[dates.date!=-1]; pbar.update(1)
    dates = dates[dates.date!=-3]; pbar.update(1)
    dates.date = dates.date.apply(datetime_from_dec_year); pbar.update(1)
    
    test = codes.merge(dates, how="left", on=["eid", "instance_index"]).assign(origin="self_reported").copy(); pbar.update(1)
    
    test["instance_index"] = test["instance_index"].astype("string"); pbar.update(1)
    test[['instance','n']] = test.instance_index.str.split("_",expand=True); pbar.update(1)
    pbar.close()
    
    return test[["eid", "origin", 'instance','n', "code", "meaning", "date"]]

In [30]:
data_geno_path = "D:/UKBiobank/Geno"
data_base_path = "D:/UKBiobank/brain/2_datasets_pre"
save_data_path = "D:/UKBiobank/ML_Stroke/Data"
data_sup1_path = "D:/UKBiobank/Sup1"
data_sup4_path = "D:/UKBiobank/Sup4"
data_sup5_path = "D:/UKBiobank/Sup5"
data_exam_path = "D:/UKBiobank/Exam"
data_path = "D:/UKBiobank/Green"

In [31]:
# load data
ukbiobank_variable_definitions = pd.read_csv("D:/UKBiobank/AF_PHENOTYPE_GTM/data/ukbiobank/ukbiobank_variable_definitions.csv")
phecode_df = pd.read_csv("D:/UKBiobank/AF_PHENOTYPE_GTM/data/ukbiobank/phecode_icd10_mappings.csv", encoding="latin_1")

In [32]:
temp_stroke = pd.read_feather(f"{save_data_path}/temp_stroke_diagnosis_image.feather")
var_df = pd.read_feather(f"{save_data_path}/temp_cmr.feather")

In [33]:
selected_columns = var_df.iloc[:,1:]
var_df_notna = var_df[selected_columns.notna().all(axis=1)].reset_index(drop=True)

In [34]:
len(temp_stroke.columns.values)

43

In [35]:
model_vis_df = temp_stroke.iloc[:, [0,1,4,7,10,13,16,19,22,25,28,31,34,37,40]]

In [36]:
model_vis_df.columns

Index(['eid', 'ALL Stroke_event', 'Self report - Stroke_event',
       'Self report - Subarachnoid haemorrhage_event',
       'Self report - Brain haemorrhage_event',
       'Self report - Ischaemic stroke_event',
       'ICD 9 - Subarachnoid haemorrhage_event',
       'ICD 9 - Intracerebral haemorrhage_event',
       'ICD 9 - Occlusion of cerebral arteries_event',
       'ICD 9 - Acute, but ill-defined, cerebrovascular disease_event',
       'ICD 10 - Subarachnoid haemorrhage_event',
       'ICD 10 - Intracerebral haemorrhage_event',
       'ICD 10 - Other nontraumatic intracranial haemorrhage_event',
       'ICD 10 - Cerebral infarction_event',
       'ICD 10 - Stroke, not specified as haemorrhage or infarction_event'],
      dtype='object')

In [37]:
new_columns = [model_vis_df.columns[0]] + [col.replace('_event', '') for col in model_vis_df.columns[1:]]
model_vis_df.columns = new_columns

In [38]:
# get AF definition for each participants
stroke_definition_vis = model_vis_df

# get the variables for modeling
modelling_df = var_df_notna[var_df_notna['eid'].isin(model_vis_df['eid'])].reset_index(drop=True)
modelling_df_NOID = modelling_df.drop("eid", axis=1).reset_index(drop=True)

# The number of participants with CMR examination was 37898; the number of participants with stroke was 13378; the overlap number was 642; This is nothing
# The final purpose of CMR examization was to identify clusters of participants.
# The target population was xxx? How to define the population? The stroke patients? or the general population or the healthy population?
# The UK Biobank was a study about the general population. So the clustering must be the general population with furture risk of some kind of disease?
# Clustering the general population excluding participants with disease at baseline? 


# The goal was clustering CMR in patients free of stroke before imaging visiting and find the cluster with higher risk of stroke

In [44]:
def find_indices(list1, list2):
    return [i for i, x in enumerate(list2) if x in list1]

# Extract the column positions for data that need to be log transformed
log_vars = list(ukbiobank_variable_definitions["UDI"].loc[ukbiobank_variable_definitions["log_transform"]==1].values)

all_model_vars = modelling_df_NOID.columns

log_pos = find_indices(log_vars, all_model_vars)

# Impute missing values within the dataset
imp = IterativeImputer(max_iter=10, random_state=0)

# Fit the imputer on the dataset
imp.fit(modelling_df_NOID)

# Transform the dataset (impute the missing values)
X_transformed = imp.transform(modelling_df_NOID)

for pos in log_pos:
    X_transformed[:,pos] = np.log(X_transformed[:,pos]+1-np.min(X_transformed[:,pos]))

data_df_impute = pd.DataFrame(X_transformed)
data_df_impute.columns = modelling_df_NOID.columns

# Scale and tranform the data
scaler = StandardScaler().fit(data_df_impute)
Xvis = scaler.transform(data_df_impute)

In [47]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from ugtm import runGTM

from sklearn.preprocessing import StandardScaler, MinMaxScaler



In [48]:
# 自定义负对数似然评分函数
def negative_log_likelihood_scorer(estimator, X):
    log_likelihood = estimator.score(X)
    return -log_likelihood * len(X)

class UGTMTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, k=15,m=14,regul=1,s=0.1, random_state=None):
        self.k = k
        self.regul = regul
        self.random_state = random_state
        self.m = m
        self.s = s

    def fit(self, X, y=None):
        self.ugtm_ = runGTM(X, k=self.k, random_state=self.random_state, m=self.m,regul=self.regul, s = self.s)
        return self

    def transform(self, X):
        return MinMaxScaler().fit_transform(self.ugtm_.matY.T).T


# 创建管道
pipe_ugtm = Pipeline([('ugtm', UGTMTransformer(random_state=42)), ('gmm', GaussianMixture(random_state=42))])

# 参数设置
param_grid_ugtm = {
    'ugtm__k': [16], 
    'ugtm__m': [4],  
    'ugtm__s': [0.1, 0.2, 0.3],
    'ugtm__regul':[0.0001, 0.001, 0.01,0.1],
    #'gmm__n_components': [2, 3, 4],  # 搜索 2, 3 和 4 组件的 GMM
    #'gmm__covariance_type': ['full', 'tied', 'diag', 'spherical']  # GMM 的协方差类型
}

# 执行 GridSearchCV
#grid_search_pca = GridSearchCV(pipe_pca, param_grid_pca, scoring=negative_log_likelihood_scorer, cv=10)
#grid_search_tsne = GridSearchCV(pipe_tsne, param_grid_tsne, scoring=negative_log_likelihood_scorer, cv=10)
grid_search_ugtm = GridSearchCV(pipe_ugtm, param_grid_ugtm, scoring=negative_log_likelihood_scorer, cv=10)

# 拟合模型并搜索最佳参数
#grid_search_pca.fit(X)
#grid_search_tsne.fit(X)
grid_search_ugtm.fit(Xvis)

# 输出最佳参数和对应的 NLL 得分
#print("Best parameters for PCA pipeline:", grid_search_pca.best_params_)
#print("Best NLL for PCA pipeline:", grid_search_pca.best_score_)
 
#print("Best parameters for t-SNE pipeline:", grid_search_tsne.best_params_)
#print("Best NLL for t-SNE pipeline:", grid_search_tsne.best_score_)

print("Best parameters for UGTM pipeline:", grid_search_ugtm.best_params_)
print("Best NLL for UGTM pipeline:", grid_search_ugtm.best_score_)

Best parameters for UGTM pipeline: {'ugtm__k': 16, 'ugtm__m': 4, 'ugtm__regul': 0.001, 'ugtm__s': 0.2}
Best NLL for UGTM pipeline: -5423331.65468949
